<a href="https://colab.research.google.com/github/pleabargain/ipynb_notebooks/blob/master/anthony_bui_story_generator_fails_with_key_error.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tiger Mom: Unfair Comparison Story Generator

by Anthony Bui

This is a story-like biography generator of impressive people. Under the hood, it's a Markov chain built using data of speaker profiles from conferences. This proved to be a good dataset because it is very succinct and factual, in essence highlighting the individuals achievements in a paragraph.

Markov chain narrative techniques are stolen from [Allison Parrish's Tutorial](https://mybinder.org/v2/gh/aparrish/corpus-driven-narrative-generation/master?filepath=corpus-driven-narrative-generation.ipynb)

Data scraped and cleaned in a separate scraping script done in ruby and shell because Python scraping takes me forever. =P

## Setup: Install and import shit

In [0]:
!pip install -q markovify
!pip install -q spacy
!python -m spacy download en_core_web_sm
!pip install -q inflect

    100% |████████████████████████████████| 245kB 10.4MB/s 
    100% |████████████████████████████████| 17.3MB 1.8MB/s 
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en_core_web_sm

    You can now load the model via spacy.load('en_core_web_sm')



In [0]:
import markovify
from google.colab import files
import os
import random
import spacy
nlp = spacy.load('en_core_web_sm')

## Upload data file and parse sentences with spaCy

In [0]:
uploaded = files.upload()
all_files = [(name, os.path.getmtime(name)) for name in os.listdir()]
latest_file = sorted(all_files, key=lambda x: -x[1])[0][0]

sentences = []
for line in open(latest_file):
    line = line.strip()
    sentences.append(line)
    
sentences[:10]

Saving teset.txt to teset (1).txt


["'Andrew joined Philo as the company’s CEO in 2014. Before assuming the role of CEO, Andrew served on Philo’s board, mentored Philo’s co-founders, and was one of the company’s first investors. Prior to Philo, Andrew was on the founding team of Facebook, and the social design of products has been a persistent interest. After Facebook, Andrew was an Entrepreneur in Residence at two of Philo’s investors, New Enterprise Associates (NEA) and Flybridge Partners. He continues to be an active early-stage angel investor and advisor. Andrew holds a Bachelor’s Degree in Computer Science and a Master’s Degree in Education from Harvard University.',",
 "'President of NCSOFT, responsible for managing the organization and planning the company’s globalization strategy. NCSOFT is one of the most successful online game company globally, which started developing and servicing network based online multiplayer games in 1997. Her early work focused on MLNN in medical imaging applications. As a VP of Commun

In [0]:
split_sentences = [list(nlp(sentence).sents) for sentence in sentences]

## Create Markov Models for Beginning / Middle / Ending

In [0]:
beginnings = []
middles = []
endings = []

for splits in split_sentences:
  if not splits:
    continue
    
  beginnings.append(str(splits[0]))
  endings.append(str(splits[-1]))
  middles.extend([str(split) for split in splits[1:-2]])

In [0]:
beginnings_gen = markovify.NewlineText("\n".join(beginnings))
middles_gen = markovify.NewlineText("\n".join(middles))
endings_gen = markovify.NewlineText("\n".join(endings))
# combo = markovify.combine([beginnings_gen, middles_gen, endings_gen], [10, 1, 10])

KeyError: ignored

## Generate Stories!

In [0]:
def generate_name():
  unisex_names = ["Avery", "Riley", "Jordan", "Angel", "Parker", "Sawyer", "Peyton", "Quinn", "Blake", "Hayden", "Taylor", "Alexis", "Rowan", "Charlie", "Emerson", "Finley", "River", "Ariel", "Emery", "Morgan", "Elliot", "London", "Eden", "Elliott", "Karter", "Dakota", "Reese", "Zion", "Remington", "Payton", "Amari", "Phoenix", "Kendall", "Harley", "Rylan", "Marley", "Dallas", "Skyler", "Spencer", "Sage", "Kyrie", "Lyric", "Ellis", "Rory", "Remi", "Justice", "Ali", "Haven", "Tatum", "Kamryn", "Charlie", "Finley", "Skyler", "Justice", "Royal", "Lennon", "Oakley", "Armani", "Azariah", "Landry", "Arden", "Arrow", "Auden", "Baker", "Bellamy", "Blair", "Bowie", "Briar", "Channing", "Charleston", "Cleo", "Devon", "Drew", "Egypt", "Ellison", "Ever", "Everest", "Frankie", "Gentry", "Grey", "Halo", "Holland", "Indigo", "Jules", "Kingsley", "Landry", "Lennox", "Linden", "Memphis", "Milan", "Miller", "Monroe", "Navy", "Oakley", "Ocean", "Quincy", "Raleigh", "Rebel", "Remy", "Rio", "Ripley", "Sailor", "Scout", "Shay", "Shiloh", "Sutton", "Tobin", "Wilder", "Wren", "Zephyr", "Adlai", "Amory", "Austen", "Avis", "Bergen", "Blue", "Breslin", "Brighton", "Callaway", "Campbell", "Carmel", "Clancy", "Clarke", "Denver", "Gentry", "Hollis", "Honor", "Indiana", "Jazz", "Jules", "Kirby", "Lake", "Laramie", "Marlo", "Mercury", "Merit", "Murphy", "Nicky", "Ocean", "Oswin", "Peace", "Perry", "Poet", "Ramsey", "Reilly", "Revel", "Ridley", "Robin", "Rumi", "Seneca", "Sidney", "Storm", "Tennessee", "Tennyson", "Texas", "Timber", "Tory", "Valentine", "Wisdom", "Wynn"]
  name = random.choice(unisex_names) + " " + random.choice(unisex_names)
  return name

def replace_person(text, name):
  doc = nlp(text)
  
  for ent in doc.ents:
    if ent.label_ == "PERSON":
      text = text.replace(ent.text, name)
      
  return text

def replace_pronouns(text, pronoun):
  if pronoun == "MALE":
    text = text.replace(" she ", " he ")
    text = text.replace(" her ", " his ")
    text = text.replace(" Ms ", " Mr ")
    text = text.replace(" Mrs ", " Mr ")
  else:
    text = text.replace(" he ", " she ")
    text = text.replace(" his ", " her ")
    text = text.replace(" Mr ", " Ms ")

  return text


In [0]:
def generate_story():
  story = [beginnings_gen.make_sentence(), middles_gen.make_sentence(), endings_gen.make_sentence()]
  
  name = generate_name()
  story = [replace_person(line, name) for line in story]

  pronoun = random.choice(["MALE", "FEMALE"])
  story = [replace_pronouns(line, pronoun) for line in story]

  return "\n".join(story)

print(generate_story())

stories = [generate_story() for i in range(1, 10)]

Wisdom Arden is a hacker, investor and serial entrepreneur and CEO/Co-Founder of FBE
After earning a Political Science B.A. from the University of Virginia, where she focused on high tech and innovative business startups.
Mr. Wisdom Arden received her Bachelor of Arts in mass communications and Bachelor of Science in Economics, with concentrations in Finance from Rutgers University.


In [0]:
import json
with open('stories.txt', 'w') as f:
  f.write(json.dumps(stories))

files.download('stories.txt')

## DEBUG THINGS

In [0]:
text = "Russ’ work has focused on exploring innovative products and services including CBS All Access and CBSN, and managing its global business strategy."
doc = nlp(text)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

print("---")

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
          token.shape_, token.is_alpha, token.is_stop)

CBS 79 82 ORG
CBSN 98 102 ORG
---
Russ’ russ’ NOUN NN compound Xxxx’ False False
work work NOUN NN nsubj xxxx True False
has have VERB VBZ aux xxx True True
focused focus VERB VBN ROOT xxxx True False
on on ADP IN prep xx True True
exploring explore VERB VBG pcomp xxxx True False
innovative innovative ADJ JJ amod xxxx True False
products product NOUN NNS dobj xxxx True False
and and CCONJ CC cc xxx True True
services service NOUN NNS conj xxxx True False
including include VERB VBG prep xxxx True False
CBS cbs PROPN NNP pobj XXX True False
All all PROPN NNP det Xxx True False
Access access PROPN NNP dobj Xxxxx True False
and and CCONJ CC cc xxx True True
CBSN cbsn PROPN NNP conj XXXX True False
, , PUNCT , punct , False False
and and CCONJ CC cc xxx True True
managing manage VERB VBG conj xxxx True False
its -PRON- ADJ PRP$ poss xxx True True
global global ADJ JJ amod xxxx True False
business business NOUN NN compound xxxx True False
strategy strategy NOUN NN dobj xxxx True False
. . PU

In [0]:
!pip install -q tracery
import tracery
from tracery.modifiers import base_english

In [0]:
name = generate_name()

accusation_rules = {
    'name': name,
    'origin': [
      "Why can't you be more like your cousin, #name#?",
      '#hello.capitalize#, #location#!'   
    ],
    'hello': ['hello', 'greetings', 'howdy', 'hey'],
    'location': ['world', 'solar system', 'galaxy', 'universe']
}

grammar = tracery.Grammar(accusation_rules)
grammar.add_modifiers(base_english)
print(grammar.flatten("#origin#"))

Greetings, solar system!
